In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
import re
from sklearn.pipeline import Pipeline
import nltk
from nltk.stem.snowball import SnowballStemmer
from sklearn.metrics import f1_score
from nltk.corpus import stopwords
import pickle
import tweepy
import sys
from datetime import datetime

user = 'politicalfacts'


# create connection with Twitter API

CONSUMER_KEY = 'J3uhwWHdmSt3uD69ry8r2kc3B'
CONSUMER_SECRET = 'LeuM7dVfFbXH0bc5fFkZIxlNiUXyjIx4Kcjh58HeWs2TrLijDx'
ACCESS_TOKEN = '327497511-5dNjiGaTQHASljoxNr1qlMOlHrrSB21HBFBDjx2E'
ACCESS_TOKEN_SECRET = 'tHLqwlhHYJWptW3femyq0rbMW6ZItu5yLQ3DFGJqJ8Xlg'

auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = tweepy.API(auth)



# download user's tweets

stuff = api.user_timeline(screen_name = user, count = 100, include_rts = True, tweet_mode="extended")
tweets = []

for tweet in stuff:
    tweet._json['user_id'] = tweet._json['user']['id']
    if len(tweet._json['entities']['urls']) != 0:
        tweet._json['url'] = tweet._json['entities']['urls'][0]['expanded_url']
    else:
         tweet._json['url'] = None
    del tweet._json['user'], tweet._json['entities']

    tweets.append(tweet._json)
        
tweets = pd.DataFrame.from_dict(tweets)


# download user's features

user = api.get_user(user)

user_features = pd.DataFrame([[user.id,
                    user.name,
                    user.screen_name,
                    user.statuses_count,
                    user.followers_count,
                    user.friends_count,
                    user.favourites_count,
                    user.listed_count,
                    user.url,
                    user.lang,
                    user.time_zone,
                    user.location,
                    user.default_profile,
                    user.default_profile_image,
                    user.geo_enabled,
                    user.profile_image_url,
                    user.profile_use_background_image,
                    user.profile_background_image_url_https,
                    user.profile_text_color,
                    user.profile_image_url_https,
                    user.profile_sidebar_border_color,
                    user.profile_background_tile,
                    user.profile_sidebar_fill_color,
                    user.profile_background_image_url,
                    user.profile_background_color,
                    user.profile_link_color,
                    user.utc_offset,
                    user.is_translator,
                    user.follow_request_sent,
                    user.protected,
                    user.verified,
                    user.notifications,
                    user.description,
                    user.contributors_enabled,
                    user.following,
                    user.created_at]],
                    columns=["id","name","screen_name","statuses_count","followers_count","friends_count","favourites_count","listed_count","url","lang","time_zone","location","default_profile","default_profile_image","geo_enabled","profile_image_url","profile_use_background_image","profile_background_image_url_https","profile_text_color","profile_image_url_https","profile_sidebar_border_color","profile_background_tile","profile_sidebar_fill_color","profile_background_image_url","profile_background_color","profile_link_color","utc_offset","is_translator","follow_request_sent","protected","verified","notifications","description","contributors_enabled","following","created_at"]
                  )

## Context

In [2]:
# compute context score

porn_words = pd.read_csv('../scripts/data/porn/filtered_main_words.csv', sep=',')
prop_words = pd.read_csv('../scripts/data/propaganda/filtered_main_words.csv', sep=',')
spam_words = pd.read_csv('../scripts/data/spam/filtered_main_words.csv', sep=',')
fake_words = pd.read_csv('../scripts/data/fake_followers/filtered_main_words.csv', sep=',')
genuine_words = pd.read_csv('../scripts/data/genuine/filtered_main_words.csv', sep=',')

def compute_score(tweets):

    user_score = pd.DataFrame(columns=['porn_words_score', 'prop_words_score', 'spam_words_score', 'fake_words_score'])

    for tweet in tweets['full_text']:
        # check for words in main_words and compute the scores for each tweet and for each category
        mask = np.in1d(porn_words.word, tweet.split())
        porn_score = porn_words.loc[mask]['score'].values.sum()
        mask = np.in1d(prop_words.word, tweet.split())
        prop_score = prop_words.loc[mask]['score'].values.sum()
        mask = np.in1d(spam_words.word, tweet.split())
        spam_score = spam_words.loc[mask]['score'].values.sum()
        mask = np.in1d(fake_words.word, tweet.split())
        fake_score = fake_words.loc[mask]['score'].values.sum()
        mask = np.in1d(genuine_words.word, tweet.split())
        genuine_score = genuine_words.loc[mask]['score'].values.sum()
        
        user_score = user_score.append(pd.DataFrame({'porn_words_score': porn_score, 'prop_words_score': prop_score, 'spam_words_score': spam_score,'fake_words_score': fake_score,'genuine_words_score': genuine_score}, index=[0]),sort=False, ignore_index=True)

    return user_score


if len(tweets) > 0:
    # sum all the scores of each category
    user_score = compute_score(tweets).sum()
    scores = np.divide(user_score,len(tweets))
else:
    scores = pd.DataFrame({'porn_words_score': 0, 'prop_words_score': 0, 'spam_words_score': 0, 'fake_words_score': 0, 'genuine_words_score': 0}, index=[0])


In [3]:
scores = pd.DataFrame(scores).T

In [4]:
# compute intradistances

def compute_centroid(tf_idf):

    center = tf_idf.sum(axis=1)/tf_idf.shape[0]
    return center

In [5]:
def dist_from_centroid(tf_idf, centroid):
    
    distances = []
    for elem in tf_idf:
        distances.append(np.linalg.norm(tf_idf - centroid))
    return distances

In [6]:
def wss(id, tweets_df, is_tweet = 1):
    
    if is_tweet == 1:
        # get tweets per id
        vector = tweets_df['full_text']
        n_vectors = len(vector)
    elif is_tweet == 0:
        # get domains per id
        vector = tweets_df['url']
        vector = vector.fillna('').astype(str)
        for i in range(len(vector)):
            vector.iloc[i] = urlparse(vector.iloc[i]).netloc
        n_vectors = len(vector)
    else:
        print ('Invalid Input')

    transformer = TfidfVectorizer(smooth_idf=True)
    tf_idf = transformer.fit_transform(vector).todense()
    
    centroid = compute_centroid(tf_idf)
    distances = dist_from_centroid(tf_idf, centroid)
    avg_dist = np.asarray(distances).sum()/n_vectors
    
    return avg_dist

In [7]:
def intradistances():
    try:
        tw = (wss(user, tweets, 1))
    except:
        tw = 0
        
    try:
        url = (wss(user, tweets, 0))
    except:
        url = 0
        
    return  pd.DataFrame({'tweets_intra': tw, 'url_intra': 0}, index=[0])

In [8]:
intradistances = intradistances()

## Descriptive Features

In [9]:
#collect descriptive features

def describe_tweets(tweets):
    
    ret_perc, media_perc, url_perc, quote_perc = tweet_perc(tweets)
    
    avg_len, avg_ret, avg_fav, avg_hash = tweet_desc(tweets, 'avg')
    max_len, max_ret, max_fav, max_hash = tweet_desc(tweets, 'max')
    min_len, min_ret, min_fav, min_hash = tweet_desc(tweets, 'min')
    
    freq = tweet_freq(tweets)

    frame = np.array([avg_len, max_len, min_len, avg_ret, max_ret, min_ret, avg_fav, max_fav, min_fav, avg_hash, max_hash, min_hash, freq, ret_perc, media_perc, url_perc, quote_perc])
   
    desc_features = pd.DataFrame({'avg_len': avg_len, 'max_len': max_len, 'min_len': min_len, 'avg_ret': avg_ret, 'max_ret': max_ret, 'min_ret': min_ret, 'avg_fav': avg_fav, 'max_fav': max_fav, 'min_fav': min_fav, 'avg_hash' : avg_hash, 'max_hash' : max_hash, 'min_hash' : max_hash,'freq': freq, 'ret_perc': ret_perc, 'media_perc': media_perc, 'url_perc': url_perc, 'quote_perc': quote_perc}, index=[0])
    
    return desc_features

In [10]:
def tweet_perc(tweets):
    
    try:
        ret_perc = np.invert(tweets.retweeted_status.isnull()).sum()/len(tweets)
    except:
        ret_perc = 0
    try:
        media_perc = np.invert(tweets.extended_entities.isnull()).sum()/len(tweets)
    except:
        media_perc = 0
    try:
        url_perc = np.invert(tweets.url.isnull()).sum()/len(tweets)
    except:
        url_perc = 0
    try:
        quote_perc = tweets.is_quote_status.sum()/len(tweets)
    except:
        quote_perc = 0
    
    return ret_perc, media_perc, url_perc, quote_perc

In [11]:
def hashtag_count(tweets):
    
    occurrences = []
    for tweet in tweets:
        occurrences.append(tweet.count('#'))
        
    return occurrences

In [12]:
def tweet_desc(tweets, metric):
    
    tweets_lenght = tweets['full_text'].apply(lambda x: len(x))
    
    if metric == 'avg':
        ret = np.mean(tweets.retweet_count)
        lenght = np.mean(tweets_lenght)
        fav = np.mean(tweets.favorite_count)
        hashtag = np.mean(hashtag_count(tweets['full_text']))
    elif metric == 'max':
        ret = max(tweets.retweet_count)
        lenght = max(tweets_lenght)
        fav = max(tweets.favorite_count)
        hashtag = max(hashtag_count(tweets['full_text']))
    elif metric == 'min':
        ret = min(tweets.retweet_count)
        lenght = min(tweets_lenght)
        fav = min(tweets.favorite_count)
        hashtag = min(hashtag_count(tweets['full_text']))

    return lenght, ret, fav, hashtag

In [13]:
def tweet_freq(tweets):
    
    dates = list(tweets.created_at)
    
    last = dates[0]
    d = last[8:10]
    m = last[4:7]
    y = last[-4:]
    date = d + ' ' + m + ' ' + y
    last = datetime.strptime(date, '%d %b %Y')
    
    first = dates[-1]
    d = first[8:10]
    m = first[4:7]
    y = first[-4:]
    date = d + ' ' + m + ' ' + y
    first = datetime.strptime(date, '%d %b %Y')
    
    delta = (last - first).days + 1
    freq = len(tweets)/delta
    
    return freq

In [14]:
def describe(tweets_df):
    
    tweets = tweets_df
    
    if len(tweets) > 0:
        # sum all the scores of each category
        features = describe_tweets(tweets)
    else:
        features = pd.DataFrame({'avg_len': 0, 'max_len': 0, 'min_len': 0,'avg_ret': 0, 'max_ret': 0, 'min_ret': 0, 'avg_fav': 0, 'max_fav': 0, 'min_fav': 0, 'avg_hash': 0, 'max_hash': 0, 'min_hash': 0, 'freq': 0, 'ret_perc': 0, 'media_perc': 0, 'url_perc': 0, 'quote_perc':0}, index=[0])
    
    # return the average scores of each user
    return features

In [15]:
features = describe(tweets)

In [16]:
full = pd.concat([user_features, intradistances, scores, features], axis=1)

In [17]:
# RF classification

def oldness(x):
    x = str(x)
    if x[0] == '2':
        return 2018 - int(x[:4])
    else:
        return 2018 - int(x[-4:])
    
full = full.drop(columns=['contributors_enabled', 'follow_request_sent', 'following', 'profile_background_image_url', 'profile_background_image_url_https', 'profile_image_url', 'profile_image_url_https', 'time_zone', 'utc_offset'])
full = full.drop(columns=['default_profile_image','is_translator', 'geo_enabled', 'location', 'notifications', 'profile_background_tile', 'protected', 'verified'])
full['default_profile'] = full['default_profile'].fillna(full['default_profile'].mode()[0])
full['description'] = full['description'].fillna('')
full['description_len'] = full['description'].apply(lambda x: len(x))
full = full.drop(columns=['description'])
full['name'] = full['name'].fillna('')
full['name_len'] = full['name'].apply(lambda x: len(x))
full = full.drop(columns=['name'])
full['screen_name'] = full['screen_name'].fillna('')
full['screen_name_len'] = full['screen_name'].apply(lambda x: len(x))
full = full.drop(columns=['screen_name'])
full = full.drop(columns=['lang'])
full['age'] = full['created_at'].apply(lambda x: oldness(x))
full = full.drop(columns=['created_at'])
full = full.drop(columns=['id'])
full['profile_use_background_image'] = full['profile_use_background_image'].fillna(full['profile_use_background_image'].mode()[0])
full['url'] = (full['url'].notnull()).astype(int)
full = full[['avg_fav', 'avg_hash', 'avg_len', 'avg_ret', 'default_profile',
       'fake_words_score', 'favourites_count', 'followers_count', 'freq',
       'friends_count', 'genuine_words_score', 'listed_count', 'max_fav',
       'max_hash', 'max_len', 'max_ret', 'media_perc', 'min_fav', 'min_hash',
       'min_len', 'min_ret', 'porn_words_score',
       'profile_use_background_image', 'prop_words_score', 'quote_perc',
       'ret_perc', 'spam_words_score', 'statuses_count',
       'tweets_intra', 'url', 'url_intra', 'url_perc',
       'description_len', 'name_len', 'screen_name_len', 'age']]

In [18]:
rf = pickle.load(open('../scripts/rf.model', 'rb'))

In [19]:
rf_scores = rf.predict_proba(full)

In [30]:
# BoN classification

full.drop(columns=['porn_words_score', 'prop_words_score', 'spam_words_score', 'fake_words_score', 'genuine_words_score'], inplace=True)

# predict bot or not

bon = pickle.load(open('../scripts/bot_or_not.model', 'rb'))
bon_scores = bon.predict_proba(full)

bon_scores = pd.DataFrame(bon_scores, columns=['bon_4', 'bon_3'])

bon_scores['bon_0'] = bon_scores['bon_3']/4
bon_scores['bon_1'] = bon_scores['bon_3']/4
bon_scores['bon_2'] = bon_scores['bon_3']/4
bon_scores['bon_3'] = bon_scores['bon_3']/4

bon_scores = bon_scores.reindex(columns=['bon_0', 'bon_1', 'bon_2', 'bon_3', 'bon_4'])

In [35]:
# NB classification

tweets = tweets['full_text']

In [36]:
# initialize pipeline

stemmer = SnowballStemmer("english", ignore_stopwords=True)

class StemmedCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(CountVectorizer, self).build_analyzer()
        return lambda doc:(stemmer.stem(w) for w in analyzer(doc))



# load model

nb = pickle.load( open( "nb.model", "rb" ) )


# define preprocess functions

def remove_rt(x):
    if 'RT @' in x:
        try:
            return x[x.find(':')+2:]
        except:
            return x
    else:
        return x


stop_words = stopwords.words('english')

def remove_stop(x):
    return [word for word in x.split() if word not in stop_words]


# preprocess tweets

tweets = tweets.apply(lambda x: remove_rt(x))
tweets = tweets.apply(lambda x: re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', x))
tweets = tweets.apply(lambda x: re.sub(r'[^\w\s]','',x))
tweets = tweets.apply(lambda x: x.lower())
tweets = tweets.apply(lambda x: remove_stop(x))
tweets = tweets.astype(str)
tweets = tweets[tweets!='[]']


# perform predictions over tweets

pred = nb.predict_proba(tweets)

# return average of predictions

nb_scores = np.mean(pred, axis=0)

[0.06428571428571428,
 0.06428571428571428,
 0.06428571428571428,
 0.06428571428571428,
 0.7428571428571429]

In [51]:
# LR final prediction

proba = pd.DataFrame(np.array(bon_scores)[0].tolist() + nb_scores.tolist() + rf_scores[0].tolist()).T

In [52]:
proba

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0.064286,0.064286,0.064286,0.064286,0.742857,0.023694,0.677967,0.215173,0.002972,0.080194,0.1,0.0,0.5,0.2,0.2


In [53]:
lr = pickle.load(open('../scripts/lr.model', 'rb'))

In [54]:
print(lr.predict_proba(proba).tolist()[0])

[0.005767441991484693, 0.23383913232371148, 0.25637080412642665, 0.029104485202460582, 0.4749181363559166]


In [56]:
proba = proba * [1.19, 2.13, 0.96, 1, 1.12, 2.66, 0, 1.8, 2.88, 1.77]
proba[0] = (proba[0] + proba[5]) / 2
proba[1] = (proba[1] + proba[6]) / 2
proba[2] = (proba[2] + proba[7]) / 2
proba[3] = (proba[3] + proba[8]) / 2
proba[4] = (proba[4] + proba[9]) / 2
proba = proba.drop(columns=[5,6,7,8,9])

In [62]:
proba.iloc[0].tolist()

[0.27304885486418223,
 0.36101736228004144,
 0.6366415533218266,
 0.6487430091919153,
 0.19945440878089352]

In [2]:
lr = pickle.load(open('../scripts/lr.model', 'rb'))

In [5]:
lr.coef_.shape

(5, 10)

In [6]:
coeff = []
for j in range (0,9):
    val = 0
    for i in range (0,4):
        val += lr.coef_[i][j]
    coeff.append(val/5)

In [7]:
coeff

[0.6733607956695952,
 -0.2811991430202746,
 -0.5576706205492699,
 0.2853986450380394,
 -0.12494548246582574,
 -0.02120902048393063,
 0.8545408071848849,
 0.6892520659239165,
 -0.11664937459305129]